# Center of Mass
## a worked example

We start from a file that has CSD data. 

In [1]:
push!(LOAD_PATH, ".")


4-element Array{String,1}:
 "@"
 "@v#.#"
 "@stdlib"
 "."

In [5]:
# Example file. Put your file here. It can be an hdf5 or jld file. Custom files, adecuatelly formated,
# can also work.
nombre="/home/karel/BRWFiles/Facilitada/Completo_19115s1cut_single_event_preproc.h5"

"/home/karel/BRWFiles/Facilitada/Completo_19115s1cut_single_event_preproc.h5"

In [7]:
esjld=false
if nombre[end-2:end]=="jld"
 esjld=true
end

In [8]:
# put the adecuate helper library for your type of file here, or use the condicional above.
if esjld
    using JLD
else
    using HDF5
end
using Statistics
using CentrosdeMasa #Module with all function necesary

┌ Info: Precompiling CentrosdeMasa [top-level]
└ @ Base loading.jl:1260


In [9]:
# open depending on file type.
if esjld
arx=load(nombre)
else
    arx=h5open(nombre)
end
#Checa como diablos se llama el CSD en cada Archivo


HDF5 data file: /home/karel/BRWFiles/Facilitada/Completo_19115s1cut_single_event_preproc.h5

In [10]:
# This is only to help load the CSD data. Use your own data name here.
if esjld
csda=arx["CSDALindenberg"]
else
    csda=read(arx["CSDALindenberg"])
end
    #csd=csddict["Trancazo_1"]+csddict["Trancazo_2"]+csddict["Trancazo_3"];
#csd=csd./3;

In [11]:
(ancho,alto,nmax)=size(csda)

(64, 64, 2000)

In [12]:
(mincsd,maxcsd)=extrema(csda)

(-852.884f0, 2565.3699f0)

This is a posible threshold value, 0.01 of the largest deviation from zero. It is not usefull for Electrical Stimulated Activity, since saturated electrodes will give signals that are out of normal ranges.

In [13]:
ϵ=max(abs(mincsd),abs(maxcsd))/100

25.653698f0

Another threshold can be obtained from the standard deviation from all the data, which includes the non active electrodes.

In [14]:
scsd=std(csda)

25.071459f0

In [15]:
#= We can multiply the standard deviation by a factor larger than one in order to have more sepparated disjoint
components. If our noice level is low, we can do the the opposite and choose a factor smaller than one. 
=#
factor=1.0
umbrsep=factor*scsd
umbrsepnota=round(umbrsep, digits=4)
(CMP, CMN)=ObtenComponentesyCM(csda,1,nmax,umbrsep);

In [16]:
nota="Disjoint components where obtained with $factor stand. dev from the CSD data, the nominal value being
 $umbrsepnota a.u."

"Disjoint components where obtained with 1.0 stand. dev from the CSD data, the nominal value being\n 25.0715 a.u."

In [20]:
# It is better to use JLD (Julia's Native data)  to save
using JLD
# output name
nomCM=nombre[1:end-4]*"-CM.jld"

save(nomCM, "CMP", CMP, "CMN", CMN, "notacm", nota)